# dropdown menu lineweaver-burk inhibitor troubleshooting
__objective__: re-purpose drop down menu code from michaelis-menten inhibitor data to lineweaver-burk inhibition plots

__approach__:
1. sub michaelis-menten functions out with lb inhibitor functions
* re-label axes and re-adjust limits
* visualize

In [ ]:
# allow inline plotting of bokeh apps
from bokeh.io import output_notebook
output_notebook()

In [ ]:
# import libraries
import os
import sys
import numpy as np

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider, Label, Span, Select
from bokeh.plotting import figure, output_file, show, ColumnDataSource

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from pharmaplot import mm

## convert older lwb functions to 1/[s]

In [ ]:
import matplotlib.pyplot as plt 

def lwb_competitive(inverse_substrate: float, vmax: float=1., km: float=5., ki: float=5., conc_i: float=5)-> float:
    """
    calculate 1/v0 based on substrate for competitive inhibitors
    """
    return ((km*(1+(conc_i/ki)))/vmax)*inverse_substrate+(1/vmax)

def lwb_noncompetitive(inverse_substrate: float, vmax: float=1., km: float=5., ki: float=5., conc_i: float=5)-> float:
    """
    calculate 1/v0 based on substrate for noncompetitive inhibitors
    """
    return (km/vmax)*((1+(conc_i/ki))* inverse_substrate)+((1+(conc_i/ki))/vmax)

def lwb_uncompetitive(inverse_substrate: float, vmax: float=1., km: float=5., ki: float=5., conc_i: float=5)-> float:
    """
    calculate 1/v0 based on substrate for uncompetitive inhibitors
    """
    return (km/vmax)*inverse_substrate +((1+(conc_i/ki))/vmax)

def plot_lwb_inhibition(substrate_concs: np.array, inhib_func, inhibitor_concs: np.array=np.linspace(0, 10, num=4),
                       vmax: float=1., km: float=5., ki: float=5., xlimits=(-0.5,1), ylimits=(-0.5,10))-> float:
    """
    plots initial velocity for multiple doses of inhibitor on the same set of axes
    """
    color_map = ['gray', 'blue', 'mediumblue', 'navy']
    
    for i, conc in enumerate(inhibitor_concs):
        # prepare data for plotting
        x = substrate_concs
        y = inhib_func(substrate_concs, ki=ki, conc_i=conc)
        
        # find the X-intercept by simulation
        mm_fit = np.array([-1/km, x.max()])
        y_fit = inhib_func(mm_fit, ki=ki, conc_i=conc)
        
        
        """ry = np.round(y_fit, decimals=1)
        x_fit = np.array([mm_fit[np.where(ry == np.round(0, decimals=1))][0], x.min()])
        y_fit = inhib_func(x_fit, ki=ki, conc_i=conc)"""
        
        # plot
        plt.scatter(x, y, color=color_map[i], label='conc. inhibitor: ' + str(i) + ' uM')
        plt.plot(mm_fit, y_fit, alpha=0.3, color=color_map[i])
        plt.ylabel('1/v0')
        plt.xlabel('1/[S]')
    
    plt.xlim(xlimits)
    plt.ylim(ylimits)
    
    plt.axvline(0, color='black', lw=0.5)
    plt.axhline(0, color='black', lw=0.5)

In [ ]:
log_range = (-1, 1.5)
s_log_space = 1 / np.logspace(log_range[0], log_range[1], base=10., num=12)

plot_lwb_inhibition(s_log_space, lwb_competitive)
plt.title('L-B Competitive Inhibition')
plt.show()

plot_lwb_inhibition(s_log_space, lwb_noncompetitive)
plt.title('L-B Noncompetitive Inhibition')
plt.show()

plot_lwb_inhibition(s_log_space, lwb_uncompetitive)
plt.title('L-B Uncompetitive Inhibition')
plt.show()

## troubleshoot visualization

In [ ]:
# generate data for plotting
log_start = -2
log_end = 1
vmax = 10
km = 1

ki = 1
conc_i = 0

x_points = 1 / np.logspace(log_start, log_end, num=12)
y_points = mm.lwb_competitive(x_points, vmax=vmax, km=km, ki=ki, conc_i=conc_i)

x_line = np.linspace(-3/km, np.max(x_points), num=5)
y_line = mm.lwb_competitive(x_line, vmax=vmax, km=km, ki=ki, conc_i=conc_i)



# set up source data and plot lines that will vary
line_source = ColumnDataSource(data=dict(x=x_line, y=y_line))
point_source = ColumnDataSource(data=dict(x=x_points, y=y_points))

plot = figure(y_range=(-0.05, 0.8), x_range=(-1.5, 4), plot_width=600, plot_height=400, 
              x_axis_label='1/[S]: substrate concentration (1/μM)',
              y_axis_label='1/initial velocity (s/μM)',
              title='Lineweaver-Burk Kinetics with Inhibition')

plot.line('x', 'y', source=line_source, line_width=3, line_alpha=0.6, color='black')
plot.circle('x', 'y', source=point_source, size=10, color='black')

# set up static line and annotations
plot.line(x_line, y_line, line_width=5, color='blue', line_alpha=0.3)
plot.circle(x_points, y_points, size=10, color='blue', line_alpha=0.3)

mytext = Label(x=2, y=0.2, text='[I] = 0 (μM)', 
               text_color="blue", text_alpha=0.5)
plot.add_layout(mytext)

# add axes lines
vline = Span(location=0, dimension='height', line_color='black', line_width=1, line_alpha=0.3)
hline = Span(location=0, dimension='width', line_color='black', line_width=1, line_alpha=0.3)
plot.renderers.extend([vline, hline])

# set up java script callback function and widgets to make plot interactive
ci_slider = Slider(start=0, end=100, value=0, step=1, title="[I] (μM)")
ki_slider = Slider(start=15, end=100, value=50, step=1, title="Ki (μM)")
inhib_select = Select(title="Inhibition Type:", value="competitive", 
                      options=["competitive", "noncompetitive", "uncompetitive"])

callback = CustomJS(args=dict(LineSource=line_source, 
                              PointSource=point_source,
                              ci=ci_slider, 
                              ki=ki_slider,
                              vmax=vmax,
                              km=km,
                              inhibType=inhib_select),
                    code="""
    const LineData = LineSource.data;
    const PointData = PointSource.data;
    const VMAX = vmax;
    const KM = km;
    const CI = ci.value;
    const KI = ki.value;
    const lx = LineData['x'];
    const ly = LineData['y'];
    const px = PointData['x'];
    const py = PointData['y'];
    const it = inhibType.value;
    
    // define functions for editing data
    function competitive(x, VMAX, KM, CI, KI){
        return ((KM*(1+(CI/KI)))/VMAX)*x+(1/VMAX);
    }
    function noncompetitive(x, VMAX, KM, CI, KI){
        return (KM/VMAX)*((1+(CI/KI))* x)+((1+(CI/KI))/VMAX);
    }
    function uncompetitive(x, VMAX, KM, CI, KI){
        return (KM/VMAX)*x +((1+(CI/KI))/VMAX);
    }
    
    // loop over line data and point data to edit
    for (var i = 0; i < lx.length; i++) {
        if (it == "competitive"){
            ly[i] = competitive(lx[i], VMAX, KM, CI, KI);
        } else if (it == "noncompetitive"){
            ly[i] = noncompetitive(lx[i], VMAX, KM, CI, KI);
        } else if (it == "uncompetitive"){
            ly[i] = uncompetitive(lx[i], VMAX, KM, CI, KI);
        }
    }
    
    for (var i = 0; i < px.length; i++) {
        if (it == "competitive"){
            py[i] = competitive(px[i], VMAX, KM, CI, KI);
        } else if (it == "noncompetitive"){
            py[i] = noncompetitive(px[i], VMAX, KM, CI, KI);
        } else if (it == "uncompetitive"){
            py[i] = uncompetitive(px[i], VMAX, KM, CI, KI);      
        }
    }
    
    // emit changes
    LineSource.change.emit();
    PointSource.change.emit();
""")

# add sliders to plot and display
ci_slider.js_on_change('value', callback)
ki_slider.js_on_change('value', callback)
inhib_select.js_on_change('value', callback)

layout = row(
    plot,
    column(ci_slider, ki_slider, inhib_select),
)

#output_file("mm.html", title="mm.py example")
show(layout)